In [1]:
AGENT_NAME = r'20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = r'20 bin PPO 500 results\binary classifier uACG results' + '/'
ATK_NAME = 'untargeted_binary_myPGD_03_mask_time_REscale_solar_and_consumption_eps_clipped_adv_obs'
CONSUMPTION_SPREAD = 0.016
CONSUMPTION_IDX = 26
SOLAR_IDX = 24
SOLAR_SPREAD = 0.004 #0.04 prev typo
MIN_OBS = 0
MAX_OBS = 1

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

from stable_baselines3 import PPO

from citylearn.data import DataSet

import pandas as pd
import numpy as np
import json

import KBMproject.utilities as utils

from tqdm import tqdm


In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
agent = PPO.load(path=f"{AGENT_NAME}")
policy_net = utils.extract_actor(agent)

In [5]:
env = utils.make_discrete_env(schema=schema,  
                        action_bins=agent.action_space[0].n,
                        seed=42)
cols = env.observation_names

In [6]:
def my_pgd_linf(model, X, y=None, loss_fn=None, epsilon:float=0.05, step:float=0.01, num_iter:int=100, 
                num_decay:int=0, decay_rate=1):
    """ Construct FGSM adversarial examples on the examples X with random restarts
    ref: https://adversarial-ml-tutorial.org/adversarial_examples/
    made for X as a single sample"""

    assert 0 < decay_rate <= 1, 'decay rate must be between 0 and 1'
    assert loss_fn is not None, 'Loss function must be provided'

    model.eval()
    
    if y is None:
        y = model(X)
        n_out = y.shape[0]
        if n_out > 1: #multiple outputs, assumes X is 1d
            _, y = torch.max(y, -1) #argmax, max returns (values, indeces)
            y = F.one_hot(y,num_classes=n_out)

    if num_decay > 0: 
        decay_iters = num_iter//num_decay
    else: #no decay
        decay_iters = num_iter

    delta = torch.zeros_like(X, requires_grad=True)
    for iter in range(num_iter):

        loss = loss_fn(reduction='none')(model(X + delta), y)
        loss.backward(torch.ones_like(loss))

        # Perform the update on delta (via the data attribute to skip the gradient tracking)
        delta.data = (delta + step*delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()
        
        if(iter%decay_iters == 0):
            step *= decay_rate
        
    return delta


In [7]:
class MaximumBifuricationWrapper(nn.Module):
    """modified to work with 1d samples
    maybe add an option dim=1 kwarg in init for compatibility?"""
    def __init__(self, base_model):
        super(MaximumBifuricationWrapper, self).__init__()
        self.base_model = base_model


    def forward(self, x):
        logits = self.base_model(x)
        lower_half, higher_half = torch.split(logits, logits.size(-1) // 2, dim=-1)
        
        # get the max of the lower and higher halves
        lower_max = torch.max(lower_half, dim=-1)[0]
        higher_max = torch.max(higher_half, dim=-1)[0]
        
        # concatenate the max of the lower and higher halves into a single tensor
        output = torch.cat((lower_max.unsqueeze(-1), higher_max.unsqueeze(-1)), dim=-1)
        return output

In [8]:
class DLLoss(nn.Module):
    """Carlini and Wagner or Difference Logits loss FOR UNTARGETED ATTACKS
    where the loss is difference between the target/clean
    logit and any other
    this version is wtitten for 1d samples, ART uses 2d"""
    def __init__(self, reduction=None):
        super(DLLoss, self).__init__()
        self.reduction = reduction

    def forward(self, logits, target_one_hot): #myPGD doesn't provide a 1 hot target...
        target_logits = torch.sum(target_one_hot * logits)
        max_non_target_logits = torch.max((1 - target_one_hot) * logits)
        loss = max_non_target_logits - target_logits

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:  #reduction is None
            return loss

Adjust epsilon for different features, the slice from 0 to 6 are temporal features and setting $\epsilon$ to 0 means that these features will not be perturbed (using torch.clamp)

In [9]:
eps_list = np.ones(agent.observation_space.shape[0])*0.03
eps_list[:6] = 0.0 #masked
#these idx are improperly normalized, so eps bust be adjusted accordingly
eps_list[SOLAR_IDX] *= SOLAR_SPREAD
eps_list[CONSUMPTION_IDX] *= CONSUMPTION_SPREAD

In [10]:
kwargs = dict(
    model=MaximumBifuricationWrapper(policy_net), 
    epsilon=torch.tensor(eps_list, device=agent.device, dtype=torch.float32),
    step=0.01,
    num_iter=100,
    num_decay=4,
    decay_rate=0.5,
    loss_fn=DLLoss
)

In [11]:
time_steps = None

obs_list = []
adv_obs_list = []
a_list = []
adv_a_list = []
asr = 0
n_features = agent.observation_space.shape[0]

observations = env.reset()
if time_steps is None:
    time_steps = env.time_steps - 1

pbar = tqdm(total=time_steps)
for step in tqdm(range(time_steps)):

    obs_list.append(observations)
    actions = agent.predict(observations, deterministic=True)
    a_list.append(actions[0])

    delta = my_pgd_linf(X=torch.from_numpy(observations).to(agent.device),
#try toggling the one hot target y based on odd/even steps to imitat the optimal adversarial policy
                                         **kwargs).cpu().detach().numpy()
    #adv_obs = observations + delta
    adv_obs = np.clip(observations + delta, MIN_OBS, MAX_OBS) #keep adv obs in obs space
    adv_obs_list.append(adv_obs)
    
    a_adv, _ = agent.predict(adv_obs, deterministic=True)
    if a_adv[0] != actions[0]:
        asr += 1

    adv_a_list.append(a_adv[0])
    observations, _, _, _ = env.step(a_adv)

    #update progress bar including MAE
    pbar.update(1)
    pbar.set_postfix({'ASR': asr/(step + 1)}, refresh=True)
    if env.done:
        break

pbar.close()
asr/=time_steps


100%|██████████| 8759/8759 [23:54<00:00,  6.10it/s, ASR=0.447]


In [12]:
kpi = utils.format_kpis(env)
display(kpi)

cost_function
annual_peak_average                      1.000000
carbon_emissions_total                   0.911016
cost_total                               0.817345
daily_one_minus_load_factor_average      1.025456
daily_peak_average                       0.939530
electricity_consumption_total            0.924670
monthly_one_minus_load_factor_average    0.982108
ramping_average                          1.187636
zero_net_energy                          1.101910
Name: District, dtype: float64

In [13]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename,
                          index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print('KPIs.csv updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print('KPIs.csv created')

KPIs.csv updated


In [14]:
df_obs = pd.DataFrame(obs_list)
df_obs.columns = cols
df_obs['a'] = np.array(a_list).flatten().tolist()
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_obs-a.csv')

In [15]:
df_obs = pd.DataFrame(adv_obs_list)
df_obs.columns = cols
df_obs['a'] = np.array(adv_a_list).flatten().tolist()
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_adv_obs-a.csv')

In [16]:
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asrs = pd.read_csv(asr_savename,
                          index_col=0)
    df_asrs[ATK_NAME] = asr
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([asr])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/ASRs.csv updated


In [17]:
kwargs_to_save = {k: v for k, v in kwargs.items() if k != 'model'} #don't save NN as json
kwargs_to_save['loss_fn'] = kwargs['loss_fn'].__name__ #replace function with a string
if not isinstance(kwargs_to_save['epsilon'], float):
    kwargs_to_save['epsilon'] = eps_list.tolist() #tensors aren't json compatible, use list
with open(SAVE_DIR+f'{ATK_NAME} parameters.json', 'w') as f:
    json.dump(kwargs_to_save, f)